In [1]:
import os
import spacy
from spacy.tokens import Doc
import sddk
import pandas as pd
import re
import matplotlib.pyplot as plt
import json
import shutil
from gensim.models import KeyedVectors
import pandas as pd
import itertools
import pickle
from collections import Counter
import nltk
import re
import requests

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
nlp = spacy.load('la_core_web_lg')

In [3]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,words_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,518,6685,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6137,66744,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,9040,135979,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,16258,153868,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3297,38011,[Meteorology/Earth sciences],exact_sciences


In [4]:
ids = list(metadata_table_long["ids"].apply(lambda x: eval(x)[0]))
ids[:10]

[705665,
 801745,
 713323,
 693148,
 769230,
 732899,
 1479057,
 670589,
 598104,
 598104]

In [5]:
filtered_vocab_df = pd.read_json("../data/filtered_vocab_df.json")
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
5018,dico,42831,120825,41832,72221,69427.25,True,True,True,"say, call, tell"
6155,omnis,33152,108530,46009,66387,63519.50,True,True,True,
2515,facio,33323,109486,40134,53953,59224.00,True,True,True,"do, make, handle"
4040,pars,32591,90667,43593,51249,54525.00,True,True,True,part
6769,habeo,30884,87225,38386,57791,53571.50,True,True,True,"have, hold, possess, consider, think"
364,possum,23869,79572,36898,72892,53307.75,True,True,True,"be able, can"
897,res,27869,68790,27473,49014,43286.50,True,True,True,suddenly
4810,suus,23716,58975,33267,52441,42099.75,True,True,True,"his own, her own, its own, their own"
2392,uideo,21389,60143,28355,54288,41043.75,True,True,True,
1790,magnus,23446,61213,29708,39211,38394.50,True,True,True,"big, great"


In [6]:
vocab_list = filtered_vocab_df["word"].tolist()

In [7]:
filtered_vocab_df[filtered_vocab_df["word"]=="liber"]

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
4628,liber,22941,36345,17863,26167,25829.0,True,True,True,"book, volume, inner bark of a tree, book, book"


In [8]:
len(filtered_vocab_df)

6332

In [9]:
s = sddk.cloudSession(provider="sciencedata.dk", shared_folder_name="TOME/DATA/NOSCEMUS", owner="kase@zcu.cz")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/TOME/DATA/NOSCEMUS/


In [10]:
id = ids[0]
id

705665

In [11]:
def get_vocab_data(id, vocab_list):
    doc_vocab_instances = []
    sents_data = pickle.load(open("../data/large_data/sents_data/" + str(id) + ".pickle", "rb"))
    for sent_n, (sent_text, sent_data) in enumerate(sents_data):
        for wordform, lemma, tag, position in sent_data:
            if (lemma in vocab_list) & (tag in ["NOUN", "PROPN", "ADJ", "VERB"]):
                doc_vocab_instances.append((lemma, wordform, tag, id, sent_n, position))
    return doc_vocab_instances

In [12]:
doc_vocab_instances = get_vocab_data(599651, vocab_list)

In [13]:
doc_vocab_instances

[('terra', 'Terrae', 'PROPN', 599651, 0, (0, 6)),
 ('motus', 'motus', 'NOUN', 599651, 0, (7, 12)),
 ('carmen', 'Carmen', 'NOUN', 599651, 1, (0, 6)),
 ('franciscus', 'Francisco', 'ADJ', 599651, 2, (8, 17)),
 ('antonius', 'Antonio', 'PROPN', 599651, 2, (18, 25)),
 ('le', 'Le', 'PROPN', 599651, 2, (26, 28)),
 ('societas', 'Societate', 'NOUN', 599651, 2, (39, 48)),
 ('editio', 'Editio', 'NOUN', 599651, 3, (8, 14)),
 ('regio', 'Regione', 'NOUN', 599651, 4, (53, 60)),
 ('magnus', 'Magni', 'ADJ', 599651, 4, (79, 84)),
 ('terraemotus', 'Terraemotus', 'PROPN', 599651, 8, (0, 11)),
 ('obscurus', 'obscurus', 'ADJ', 599651, 9, (20, 28)),
 ('terra', 'terrae', 'NOUN', 599651, 9, (29, 35)),
 ('penetralis', 'penetralia', 'ADJ', 599651, 9, (36, 46)),
 ('uatis', 'vates', 'NOUN', 599651, 9, (47, 52)),
 ('fossa', 'fossis', 'NOUN', 599651, 9, (64, 70)),
 ('locus', 'loca', 'NOUN', 599651, 9, (77, 81)),
 ('notus', 'nota', 'ADJ', 599651, 9, (82, 86)),
 ('fodina', 'fodinis', 'NOUN', 599651, 9, (87, 94)),
 ('au

In [14]:
doc_vocab_instances = get_vocab_data(608638, vocab_list)
len(doc_vocab_instances)

31604

In [15]:
doc_vocab_instances[:100]

[('italia', 'Italiae', 'PROPN', 608638, 0, (33, 40)),
 ('medicus', 'medici', 'NOUN', 608638, 0, (41, 47)),
 ('praestantus', 'praestantissimi', 'ADJ', 608638, 0, (48, 63)),
 ('medicina', 'Medicina', 'PROPN', 608638, 0, (75, 83)),
 ('uarius', 'uariarum', 'ADJ', 608638, 0, (89, 97)),
 ('quaestio', 'quaestionum', 'NOUN', 608638, 0, (98, 109)),
 ('locus', 'locorum', 'NOUN', 608638, 0, (113, 120)),
 ('difficilis', 'difficilium', 'ADJ', 608638, 0, (136, 147)),
 ('expositio', 'expositionem', 'NOUN', 608638, 0, (148, 160)),
 ('contineo', 'continentes', 'VERB', 608638, 0, (161, 172)),
 ('omnis', 'omnibus', 'ADJ', 608638, 0, (174, 181)),
 ('uerus', 'ueram', 'ADJ', 608638, 0, (186, 191)),
 ('ars', 'artem', 'NOUN', 608638, 0, (192, 197)),
 ('exerco', 'exercere', 'VERB', 608638, 0, (198, 206)),
 ('uolo', 'uolunt', 'VERB', 608638, 0, (207, 213)),
 ('utilis', 'utiles', 'ADJ', 608638, 0, (222, 228)),
 ('antonius', 'Antonii', 'PROPN', 608638, 1, (24, 31)),
 ('musa', 'Musae', 'NOUN', 608638, 1, (32, 37))

In [16]:
def update_vocab(vocab_dict, doc_data):
    for lemma, wordform, tag, doc_id, sent_id, position  in doc_data:
        if lemma not in vocab_dict:
            vocab_dict[lemma] = {}
        if wordform not in vocab_dict[lemma]:
            vocab_dict[lemma][wordform] = []
        # append the tuple to the list of the appropriate wordform of the lemma
        vocab_dict[lemma][wordform].append((doc_id, sent_id, position, tag))
    return vocab_dict

In [34]:
%%time
vocab_instances = {}
failed = []
for id in ids[:20]:
    try:
        doc_vocab_instances = get_vocab_data(id, vocab_list)
        vocab_instances = update_vocab(vocab_instances, doc_vocab_instances)
    except:
        failed.append(id)

CPU times: user 29.2 s, sys: 133 ms, total: 29.4 s
Wall time: 29.7 s


In [35]:
vocab_instances["liber"]

{'libros': [(705665, 183, (174, 180), 'NOUN'),
  (705665, 282, (0, 6), 'NOUN'),
  (705665, 378, (62, 68), 'NOUN'),
  (713323, 35, (179, 185), 'NOUN'),
  (713323, 45, (1462, 1468), 'NOUN'),
  (713323, 56, (9, 15), 'NOUN'),
  (713323, 7140, (216, 222), 'NOUN'),
  (693148, 697, (32, 38), 'NOUN'),
  (693148, 757, (41, 47), 'NOUN'),
  (693148, 757, (131, 137), 'NOUN'),
  (693148, 904, (33, 39), 'NOUN'),
  (693148, 1012, (56, 62), 'NOUN'),
  (693148, 1097, (22, 28), 'NOUN'),
  (693148, 1108, (12, 18), 'NOUN'),
  (693148, 1169, (35, 41), 'NOUN'),
  (693148, 1278, (28, 34), 'NOUN'),
  (693148, 1341, (18, 24), 'NOUN'),
  (693148, 1380, (87, 93), 'NOUN'),
  (693148, 1460, (14, 20), 'NOUN'),
  (693148, 1471, (22, 28), 'NOUN'),
  (693148, 1477, (37, 43), 'NOUN'),
  (693148, 1576, (12, 18), 'NOUN'),
  (693148, 2224, (188, 194), 'NOUN'),
  (693148, 2234, (2, 8), 'NOUN'),
  (693148, 2386, (78, 84), 'NOUN'),
  (693148, 2416, (40, 46), 'NOUN'),
  (693148, 2439, (107, 113), 'NOUN'),
  (693148, 2501, (88

In [36]:
ids_covered_list = []
for key in vocab_instances.keys():
    wordforms_dict = vocab_instances[key]
    for wordform in wordforms_dict.keys():
        ids_covered_list.extend([tup[0] for tup in wordforms_dict[wordform]])

In [37]:
ids_covered = list(set(ids_covered_list))
len(ids_covered)

12

In [38]:
type(ids_covered[0])

int

In [ ]:
ids_failed = []
for id in ids:
    if id not in ids_covered:
        try:
            doc_vocab_instances = get_vocab_data(id, vocab_list)
            vocab_instances = update_vocab(vocab_instances, doc_vocab_instances)
        except:
            ids_failed.append(id)

In [207]:
len(vocab_instances.keys())

6202

In [208]:
with open("../data/large_data/vocab_instances.json", 'w') as f:
    json.dump(vocab_instances, f)

In [12]:
# check whether we have sentences for all documents:
f_sents_ready = os.listdir("../data/large_data/sents_text")
len(f_sents_ready)

1007

In [13]:
### choose a lemma for
lemma = "portendo"

In [14]:
id = ids[0]
metadata = metadata_table_long[metadata_table_long["id"] == id].squeeze()
metadata

Author                                                        Achrelius, Daniel
Full title                    Scientiarum magnes recitatus publice anno 1690...
In                                                                         None
Year                                                                       1690
Place                                                                   [Turku]
Publisher/Printer                                                          Wall
Era                                                                17th century
Form/Genre                                                              Oration
Discipline/Content            Mathematics, Astronomy/Astrology/Cosmography, ...
Original                                       Scientiarum magnes(Google Books)
Digital sourcebook                                                       705665
Description                   TheMagnet of Sciencesis a panegyrical oration ...
References                              

In [15]:
metadata_table_long.columns

Index(['Author', 'Full title', 'In', 'Year', 'Place', 'Publisher/Printer',
       'Era', 'Form/Genre', 'Discipline/Content', 'Original',
       'Digital sourcebook', 'Description', 'References', 'Cited in',
       'How to cite this entry', 'Internal notes', 'Of interest to',
       'Transkribus text available', 'Written by', 'Library and Signature',
       'ids', 'id', 'date_min', 'date_max', 'filename', 'file_year', 'sents_n',
       'words_n', 'disciplines_list', 'science_class'],
      dtype='object')

In [16]:
def read_specific_line(filename, line_number):
    with open(filename, 'r', encoding='utf-8') as f:
        return next(itertools.islice(f, line_number, None)).strip()
    
def get_contexts_for_lemma(lemma):
    contexts_data = vocab_instances[lemma]
    contexts_texts = []
    for wordform in contexts_data.keys():
        for tup in contexts_data[wordform]:
            try:   
                filename = "../data/large_data/sents_text/{}.txt".format(str(tup[0]))
                metadata = metadata_table_long[metadata_table_long["id"] == tup[0]].squeeze()
                context_text = read_specific_line(filename, tup[1])
                contexts_texts.append({"lemma" : lemma, "wordform": wordform, "pos" : tup[2], "sentence_text" : context_text, "doc_id" : tup[0], "sentence_i" : tup[1], "author" : metadata["Author"], "title" : metadata["Full title"], "year" : metadata["file_year"]})
            except:
                pass
    contexts_texts_df = pd.DataFrame(contexts_texts)
    contexts_texts_df.sort_values("year", ascending=False)
    return contexts_texts_df

In [17]:
lemma = "honor"
contexts_texts_df = get_contexts_for_lemma(lemma)

NameError: name 'vocab_instances' is not defined

In [183]:
len(contexts_texts_df)

122

In [184]:
contexts_texts_df.tail(10)

,lemma,wordform,pos,sentence_text,doc_id,sentence_i,author,title,year
112,honor,honoribus,NOUN,plissimisque honoribus affecit.,693148,4952,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
113,honor,honoribus,NOUN,quae nam 1569. tinuos quindecim annos cum fove...,693148,8341,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
114,honor,honoribus,NOUN,"um, quem aliquot annos gessit praeclarè; ampli...",693148,8623,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
115,honor,honoribus,NOUN,ibique biennium commoratus bingam redit magnos...,693148,9815,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
116,honor,honoribus,NOUN,quàm equus elegans & commodus quam in parte sa...,693148,12177,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
117,honor,honoribus,NOUN,"honoribus defunctus est, & Ioauni Vincentis Pi...",693148,12334,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
118,honor,honoribus,NOUN,Funecta & Bayerorum est familia multis aetatib...,693148,12886,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
119,honor,Honor,NOUN,"Membro Honor., Soc.",1479057,1127,"Agardh, Carl A.",Aphorismi botanici,1817.0
120,honor,Honor,NOUN,"Membro Honor., Soc.",1479057,2079,"Agardh, Carl A.",Aphorismi botanici,1817.0
121,honor,Honor,NOUN,"Membro Honor., Soc.",1479057,2588,"Agardh, Carl A.",Aphorismi botanici,1817.0
